# Structured output

OpenAI offers a functionality for defining a structure of the messages generated by LLMs, AutoGen enables this functionality by propagating `response_format` passed to your agents to the underlying client.

For more info on structured output, please check [here](https://platform.openai.com/docs/guides/structured-outputs)


````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://ag2ai.github.io/ag2/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o", "gpt-4o-mini"],
    },
)

/home/tvrtko/ag2/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

## Example: math reasoning

Using structured output, we can enforce chain-of-thought reasoning in the model to output an answer in a structured, step-by-step way

### Define the reasoning model

First we will define the math reasoning model. This model will indirectly force the LLM to solve the posed math problems iteratively trough math reasoning steps.

In [2]:
from pydantic import BaseModel


class Step(BaseModel):
    explanation: str
    output: str


class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

### Define chat actors

Now we can define the agents that will solve the posed math problem. 
We will keep this example simple; we will use a `UserProxyAgent` to input the math problem and an `AssistantAgent` to solve it.

The `AssistantAgent` will be constrained to solving the math problem step-by-step by using the `MathReasoning` response format we defined above.

In [3]:
llm_config = {"config_list": config_list, "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="Math_solver",
    llm_config=llm_config,
    response_format=MathReasoning,
)

### Start the chat

Let's now start the chat and prompt the assistant to solve a simple equation. The assistant agent should return a response solving the equation using a step-by-step `MathReasoning` model.

In [4]:
user_proxy.initiate_chat(assistant, message="how can I solve 8x + 7 = -23", max_turns=1, summary_method="last_msg")

User_proxy (to Math_solver):

how can I solve 8x + 7 = -23

--------------------------------------------------------------------------------
Math_solver (to User_proxy):

{"steps":[{"explanation":"Start with the given equation: 8x + 7 = -23. The goal is to isolate 'x' on one side of the equation. First, we will subtract 7 from both sides to eliminate the constant term from the left side of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Now, simplify both sides of the equation. On the left, 7 - 7 equals 0, leaving us with 8x. On the right, -23 - 7 equals -30.","output":"8x = -30"},{"explanation":"The equation 8x = -30 means 8 times x equals -30. To find the value of x, we need to divide both sides of the equation by 8 (the coefficient of x) to solve for x.","output":"x = -30 / 8"},{"explanation":"Simplify the fraction on the right side. -30 divided by 8 equals -3.75, so x = -3.75 is the solution to the equation.","output":"x = -3.75"}],"final_answer":"x = -3.75"}

-----

ChatResult(chat_id=None, chat_history=[{'content': 'how can I solve 8x + 7 = -23', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '{"steps":[{"explanation":"Start with the given equation: 8x + 7 = -23. The goal is to isolate \'x\' on one side of the equation. First, we will subtract 7 from both sides to eliminate the constant term from the left side of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Now, simplify both sides of the equation. On the left, 7 - 7 equals 0, leaving us with 8x. On the right, -23 - 7 equals -30.","output":"8x = -30"},{"explanation":"The equation 8x = -30 means 8 times x equals -30. To find the value of x, we need to divide both sides of the equation by 8 (the coefficient of x) to solve for x.","output":"x = -30 / 8"},{"explanation":"Simplify the fraction on the right side. -30 divided by 8 equals -3.75, so x = -3.75 is the solution to the equation.","output":"x = -3.75"}],"final_answer":"x = -3.75"}', 'role': 'user', 'name': 'Math_sol

## Formatting a response

When defining a `response_format`, you have the flexibility to customize how the output is parsed and presented, making it more user-friendly. To demonstrate this, we’ll add a format method to our `MathReasoning` model. This method will define the logic for transforming the raw JSON response into a more human-readable and accessible format.

### Define the reasoning model

Let’s redefine the `MathReasoning` model to include a `format` method. This method will allow the underlying client to parse the return value from the LLM into a more human-readable format. If the `format` method is not defined, the client will default to returning the model’s JSON representation, as demonstrated in the previous example.

In [5]:
from pydantic import BaseModel


class Step(BaseModel):
    explanation: str
    output: str


class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

    def format(self) -> str:
        steps_output = "\n".join(
            f"Step {i + 1}: {step.explanation}\n  Output: {step.output}" for i, step in enumerate(self.steps)
        )
        return f"{steps_output}\n\nFinal Answer: {self.final_answer}"

### Define chat actors and start the chat

The rest of the process is the same as in the previous example: define the actors and start the chat.

Observe how the Math_solver agent now communicates using the format we have defined in our `MathReasoning.format` method.

In [22]:
llm_config = {"config_list": config_list, "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="Math_solver",
    llm_config=llm_config,
    response_format=MathReasoning,
)

user_proxy.initiate_chat(assistant, message="how can I solve 4x + 8 = -40", max_turns=1, summary_method="last_msg")

User_proxy (to Math_solver):

how can I solve 4x + 8 = -40

--------------------------------------------------------------------------------
Math_solver (to User_proxy):

Step 1: First, we need to isolate the variable term (4x) on one side of the equation. We do this by subtracting 8 from both sides of the equation.
  Output: 4x + 8 - 8 = -40 - 8
Step 2: Simplifying both sides gives us a new equation: 4x = -48.
  Output: 4x = -48
Step 3: Next, we solve for x by dividing both sides of the equation by 4.
  Output: 4x/4 = -48/4
Step 4: Simplifying this gives us the solution for x: x = -12.
  Output: x = -12

Final Answer: x = -12

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'how can I solve 4x + 8 = -40', 'role': 'assistant', 'name': 'User_proxy'}, {'content': 'Step 1: First, we need to isolate the variable term (4x) on one side of the equation. We do this by subtracting 8 from both sides of the equation.\n  Output: 4x + 8 - 8 = -40 - 8\nStep 2: Simplifying both sides gives us a new equation: 4x = -48.\n  Output: 4x = -48\nStep 3: Next, we solve for x by dividing both sides of the equation by 4.\n  Output: 4x/4 = -48/4\nStep 4: Simplifying this gives us the solution for x: x = -12.\n  Output: x = -12\n\nFinal Answer: x = -12', 'role': 'user', 'name': 'Math_solver'}], summary='Step 1: First, we need to isolate the variable term (4x) on one side of the equation. We do this by subtracting 8 from both sides of the equation.\n  Output: 4x + 8 - 8 = -40 - 8\nStep 2: Simplifying both sides gives us a new equation: 4x = -48.\n  Output: 4x = -48\nStep 3: Next, we solve for x by dividing both sides of the equation 